In [2]:
import numpy as np
np.random.seed(7)
import os
import glob
import cv2
import math
import datetime
import pandas as pd
import random
import time
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

from matplotlib import pyplot as plt
%matplotlib inline

from joblib import Parallel , delayed

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [3]:
size=(64,64)

In [4]:
def process_image(img_file):
    img=cv2.imread(img_file)
    img = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )

    img=cv2.resize(img,size)
    
    return img #returning reduced image
     

In [5]:
def rotate_posture(img_file):
    # Reduce size
    img=cv2.imread(img_file)
    img = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )
    rotate = random.uniform(-10, 10)
    #rotate=10
    M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), rotate, 1)
    rotated_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    resized = cv2.resize(rotated_img, size, cv2.INTER_LINEAR)
    #return rotated_img ,resized
    return resized

In [ ]:
X_data = []
y_labels = []
start = time.time()

for j in range(10):
    
    path=os.path.join('imgs/train','c'+str(j),'*.jpg')
    files=glob.glob(path)
    
    X_data.extend(Parallel(n_jobs=2)(delayed(rotate_posture)(im_file)
                                      for im_file in files))
    y_labels.extend([j]*len(files))
    end=time.time()-start
    print("Time : % .2f seconds" %end)


Time :  55.97 seconds
Time :  89.58 seconds


In [6]:
def normalise_data(train_data,train_target):
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    train_data = train_data.reshape(train_data.shape[0],  1, 64, 64)
    train_target = np_utils.to_categorical(train_target, 10)
    train_data = train_data.astype('float32')
    train_data /= 255
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target 


In [7]:
X_data, y_labels = normalise_data(X_data, y_labels)

Train shape: (22424, 1, 64, 64)
22424 train samples


In [8]:
data=[]
for i in range(X_data.shape[0]):
    data.append([X_data[i] , y_labels[i]])
    
    

In [9]:
np.random.shuffle(data)

In [10]:
final_data=[]
final_target=[]
for each in data:
    final_data.append(each[0])
    final_target.append(each[1])
    
final_data=np.array(final_data)
final_target=np.array(final_target)
    

In [12]:
split_1 = int(0.6 * final_data.shape[0])
split_2 = int(0.8 * final_data.shape[0])
X_train = final_data[:split_1]
y_train = final_target[:split_1]

X_valid =  final_data[split_1:split_2]
y_valid = final_target[split_1:split_2]

X_test = final_data[split_2:]
y_test = final_target[split_2:]



In [13]:
batch_size = 32
n_epochs = 5

In [14]:
def createmodel():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3) , border_mode='same', init='he_normal',
                            input_shape=(1,64,64)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64,( 3, 3), border_mode='same', init='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), border_mode='same', init='he_normal'))
    model.add(MaxPooling2D(pool_size=(8, 8)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
    return model

In [15]:
model_driving = createmodel()

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_normal", input_shape=(1, 64, 64..., padding="same")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_initializer="he_normal", padding="same")`
  
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), kernel_initializer="he_normal", padding="same")`
  if sys.path[0] == '':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 64, 64)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 32, 32)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 16, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 16, 16)       73856     
__________

In [16]:
model_driving.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs,
               verbose=1, validation_data=(X_valid, y_valid))

Train on 13454 samples, validate on 4485 samples
Epoch 1/5
   64/13454 [..............................] - ETA: 2517s - loss: 12.4773 - acc: 0.0625

KeyboardInterrupt: 

In [17]:
from keras.models import model_from_json
json_file = open('my_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("my_weights.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [18]:
score = loaded_model.evaluate(X_test , y_test)

4485/4485 [==============================] - 243s   


In [19]:
score

[1.0704702941869015, 0.62274247494296753]

In [ ]:
 status = {0: 'Safe Driving',
             1: 'Unsafe Driving',
             2: 'Moderate Driving',
             3:'Unsafe Driving',
             4:'Moderate Driving',
             5:'Moderate Driving',
             6:'Unsafe Driving',
             7:'Unsafe Driving',
             8:'Unsafe Driving',
             9:'Moderate Driving',}

In [1]:
for c in status:
    train_files = ["imgs/train/" + c + "/" + f for f in os.listdir("imgs/train/" + c + "/")]
    random_file = random.choice(train_files)
    im = process_image(random_file)
    

NameError: name 'status' is not defined

In [ ]:
plt